# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

csv_path = "../output_data/cities.csv"
weather_df = pd.read_csv(csv_path)

weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint George,37.1041,-113.5841,293.68,39,0,1.79,US,1665387064
1,Castro,-24.7911,-50.0119,286.47,98,100,4.01,BR,1665387065
2,Yellowknife,62.4560,-114.3525,281.44,93,75,4.63,CA,1665386878
3,Port Alfred,-33.5906,26.8910,295.47,73,98,3.13,ZA,1665387066
4,George Town,5.4112,100.3354,305.11,72,20,5.14,MY,1665387028


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store locations
locations = weather_df[["Lat", "Lng"]]

#Weight
degree = weather_df["Humidity"].astype(float)



In [4]:
# Plot Heatmap
fig1 = gmaps.figure(center=(47.0, -9.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=degree, 
                                 dissipating=False,max_intensity = max(weather_df["Humidity"]),point_radius=3) 
                                 
                                     
                                 


# Add layer
fig1.add_layer(heat_layer)

# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

narrowed_df = weather_df.loc[(weather_df["Wind Speed"] < .8) & 
                (weather_df["Cloudiness"] == 0)].dropna()

narrowed_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,Bethel,41.3712,-73.4140,281.75,84,0,0.00,US,1665387070
78,Porto Velho,-8.7619,-63.9039,298.18,83,0,0.51,BR,1665387096
142,Portland,45.5234,-122.6762,293.73,75,0,0.45,US,1665386870
207,Fort Smith,35.3859,-94.3986,288.90,80,0,0.00,US,1665387133
349,Gwadar,25.1216,62.3254,305.10,25,0,0.58,PK,1665387202
350,Diego de Almagro,-26.3667,-70.0500,282.58,56,0,0.11,CL,1665387088
351,Luancheng,37.8792,114.6517,294.24,13,0,0.52,CN,1665387202
406,Bemidji,47.4736,-94.8803,278.66,100,0,0.00,US,1665387221
443,Wahiawā,21.5028,-158.0236,297.33,76,0,0.00,US,1665387234
531,Ovalle,-30.5983,-71.2003,284.35,83,0,0.66,CL,1665387269


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_df[["City", "Lat", "Lng","Country"]]


# Add a "Hotel Name" column 
hotel_df["Hotel Name"] = ""



/var/folders/8_/lkbh80z57fz2dyp47nrx749w0000gn/T/ipykernel_48114/3789025120.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
            "radius" : 5000,
            "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Hotel for: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save hotel name
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']


    except (KeyError, IndexError):
        print("key not found")
        
    print("-"*45)
    

print("Data Retrieval Complete")   

hotel_df

Retrieving Hotel for: Bethel.
Closest hotel in Bethel is La Quinta Inn & Suites by Wyndham Danbury.
---------------------------------------------
Retrieving Hotel for: Porto Velho.
Closest hotel in Porto Velho is Hotel Slaviero Porto Velho.
---------------------------------------------
Retrieving Hotel for: Portland.
Closest hotel in Portland is Hotel Eastlund.
---------------------------------------------
Retrieving Hotel for: Fort Smith.
Closest hotel in Fort Smith is Hampton Inn Fort Smith.
---------------------------------------------
Retrieving Hotel for: Gwadar.
Closest hotel in Gwadar is Zaver Pearl Continental Hotel.
---------------------------------------------
Retrieving Hotel for: Diego de Almagro.
Closest hotel in Diego de Almagro is Hotel Hambyll.
---------------------------------------------
Retrieving Hotel for: Luancheng.
Closest hotel in Luancheng is Shijiazhuang Qixin Homestay.
---------------------------------------------
Retrieving Hotel for: Bemidji.
Closest hotel 

,City,Lat,Lng,Country,Hotel Name
16,Bethel,41.3712,-73.4140,US,La Quinta Inn & Suites by Wyndham Danbury
78,Porto Velho,-8.7619,-63.9039,BR,Hotel Slaviero Porto Velho
142,Portland,45.5234,-122.6762,US,Hotel Eastlund
207,Fort Smith,35.3859,-94.3986,US,Hampton Inn Fort Smith
349,Gwadar,25.1216,62.3254,PK,Zaver Pearl Continental Hotel
350,Diego de Almagro,-26.3667,-70.0500,CL,Hotel Hambyll
351,Luancheng,37.8792,114.6517,CN,Shijiazhuang Qixin Homestay
406,Bemidji,47.4736,-94.8803,US,DoubleTree by Hilton Hotel Bemidji
443,Wahiawā,21.5028,-158.0236,US,The Inn At Schofield
531,Ovalle,-30.5983,-71.2003,CL,Hotel Limari


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig1.add_layer(markers)

# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))